# Proyecto 1 Mineria de datos

Se analizara el contexto economico y politico para poder encontrar estrategias apropiadas para poder invertir en la salud global

In [19]:
# Se importan librerias necesarias
import pandas as pd
import matplotlib.pyplot as plt
import sklearn as sk

In [64]:
# Se carga el archivo de datos
expenditure = pd.read_csv('Expenditure.csv', encoding='latin1')
Personnel = pd.read_csv('Personnel.csv', encoding='latin1')

In [65]:
# Primer paso de limpieza de datos
# Hacer que la primera fila sea el encabezado
expenditure.columns = expenditure.iloc[0]
expenditure = expenditure[1:]

Personnel.columns = Personnel.iloc[0]
Personnel = Personnel[1:]


In [66]:
# Segundo paso de limpieza de datos
# Cambiar el nombre de la segunda columna (La cual no tiene nombre y cuando la imprimo me muestra NaN) por Country
expenditure = expenditure.rename(columns={expenditure.columns[1]: 'Country Name'})
Personnel = Personnel.rename(columns={Personnel.columns[1]: 'Country Name'})


## Tercer paso de limpieza

In [67]:
# Remplazar parte de los valores de string en la columna Series
Personnel["Series"] = Personnel["Series"].str.replace("Health personnel: ", "")

#con regex extrae de forma unique los valores que estan entre parebtesis en series 
Series = Personnel["Series"].str.extract(r"\((.*?)\)", expand=False)

for serie in Series.unique():
    Personnel[serie] = 0
    Personnel.loc[Personnel["Series"].str.contains(serie), serie] = 1





# Se suman los value que tengan todas las mismas caracteristicas (Country Name, physician, Pharmacists, nurses, dentists, region/country/area, Year)

In [68]:
# Convertir la columna "Value" a cadena antes de aplicar métodos .str
Personnel['Value'] = Personnel['Value'].astype(str)

# Aplicar los métodos .str para reemplazar los caracteres no deseados
Personnel['Value'] = Personnel['Value'].str.replace(',', '').str.replace(' ', '').str.replace('..', '0')

# Si number es 1, dividir value / 1000
Personnel["Value"] = Personnel["Value"].astype(float)
Personnel.loc[Personnel["number"] == 1, "Value"] = Personnel["Value"] / 1000


#eliminar categorias 
Personnel = Personnel.drop(columns=["per 1000 population", "number"])


In [69]:
Personnel['Series'] = Personnel['Series'].str.replace(' (per 1000 population)', '')
Personnel['Series'] = Personnel['Series'].str.replace(' (number)', '')



categorias = Personnel["Series"].unique()
for categoria in categorias:
    Personnel[categoria] = 0
    Personnel.loc[Personnel["Series"].str.contains(categoria), categoria] = 1#

Personnel = Personnel.drop(columns=["Series"])
Personnel = Personnel.drop(columns=["Footnotes"])



In [70]:
Personnel

,Region/Country/Area,Country Name,Year,Value,Source,Physicians,Pharmacists,Nurses and midwives,Nurses and midwives personnel,Dentists
1,4,Afghanistan,2001,4.104,"World Health Organisation (WHO), Geneva, WHO G...",1,0,0,0,0
2,4,Afghanistan,2001,0.200,"World Health Organisation (WHO), Geneva, WHO G...",1,0,0,0,0
3,4,Afghanistan,2001,0.525,"World Health Organisation (WHO), Geneva, WHO G...",0,1,0,0,0
4,4,Afghanistan,2001,0.000,"World Health Organisation (WHO), Geneva, WHO G...",0,1,0,0,0
5,4,Afghanistan,2005,0.900,"World Health Organisation (WHO), Geneva, WHO G...",0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...
5880,716,Zimbabwe,2020,0.000,"World Health Organisation (WHO), Geneva, WHO G...",0,0,0,0,1
5881,716,Zimbabwe,2020,1.600,"World Health Organisation (WHO), Geneva, WHO G...",0,1,0,0,0
5882,716,Zimbabwe,2020,0.100,"World Health Organisation (WHO), Geneva, WHO G...",0,1,0,0,0
5883,716,Zimbabwe,2020,31.792,"World Health Organisation (WHO), Geneva, WHO G...",0,0,1,0,0


In [75]:

# Agrupar por las columnas mencionadas y sumar la columna 'Values'
Personnel = Personnel.groupby(['Country Name', 'Year', 'Source', 'Physicians', 'Pharmacists', 'Nurses and midwives', 'Nurses and midwives personnel', 'Dentists'])['Value'].sum().reset_index()

# Identificar la columna con menor valor y eliminarla
columnas = ['Physicians', 'Pharmacists', 'Nurses and midwives', 'Nurses and midwives personnel', 'Dentists']
columna_con_menor_valor = min(columnas, key=lambda x: Personnel[x].sum())
columnas.remove(columna_con_menor_valor)
Personnel.drop(columns=columna_con_menor_valor, inplace=True)

# Renombrar las columnas para que coincidan con el DataFrame original si es necesario
# grouped.rename(columns={'Values': 'Value'}, inplace=True)

# Ver el resultado
Personnel


,Country Name,Year,Source,Physicians,Nurses and midwives,Nurses and midwives personnel,Dentists,Value
0,Afghanistan,2001,"World Health Organisation (WHO), Geneva, WHO G...",0,0,0,0,0.525
1,Afghanistan,2001,"World Health Organisation (WHO), Geneva, WHO G...",1,0,0,0,4.304
2,Afghanistan,2005,"World Health Organisation (WHO), Geneva, WHO G...",0,1,0,0,14.930
3,Afghanistan,2005,"World Health Organisation (WHO), Geneva, WHO G...",0,1,1,0,0.600
4,Afghanistan,2005,"World Health Organisation (WHO), Geneva, WHO G...",0,0,0,0,0.900
...,...,...,...,...,...,...,...,...
3733,Zimbabwe,2020,"World Health Organisation (WHO), Geneva, WHO G...",0,0,0,1,0.234
3734,Zimbabwe,2020,"World Health Organisation (WHO), Geneva, WHO G...",0,1,0,0,31.792
3735,Zimbabwe,2020,"World Health Organisation (WHO), Geneva, WHO G...",0,1,1,0,2.000
3736,Zimbabwe,2020,"World Health Organisation (WHO), Geneva, WHO G...",0,0,0,0,1.700


In [80]:

#eliminar columna source 
Personnel = Personnel.drop(columns=["Source"])

In [83]:
expenditure

,Region/Country/Area,Country Name,Year,Series,Value,Footnotes,Source,% of GDP
1,4,Afghanistan,2005,Current health expenditure (% of GDP),9.9,Data based on calendar year (January 1 to Dece...,"World Health Organization (WHO), Geneva, WHO G...",0
2,4,Afghanistan,2010,Current health expenditure (% of GDP),8.6,Data based on calendar year (January 1 to Dece...,"World Health Organization (WHO), Geneva, WHO G...",0
3,4,Afghanistan,2015,Current health expenditure (% of GDP),10.1,Data based on calendar year (January 1 to Dece...,"World Health Organization (WHO), Geneva, WHO G...",0
4,4,Afghanistan,2018,Current health expenditure (% of GDP),14.2,Data based on calendar year (January 1 to Dece...,"World Health Organization (WHO), Geneva, WHO G...",0
5,4,Afghanistan,2019,Current health expenditure (% of GDP),14.8,Data based on calendar year (January 1 to Dece...,"World Health Organization (WHO), Geneva, WHO G...",0
...,...,...,...,...,...,...,...,...
2258,716,Zimbabwe,2010,Domestic general government health expenditure...,15.2,Data based on calendar year (January 1 to Dece...,"World Health Organization (WHO), Geneva, WHO G...",0
2259,716,Zimbabwe,2015,Domestic general government health expenditure...,7.6,Data based on calendar year (January 1 to Dece...,"World Health Organization (WHO), Geneva, WHO G...",0
2260,716,Zimbabwe,2018,Domestic general government health expenditure...,7.9,Data based on calendar year (January 1 to Dece...,"World Health Organization (WHO), Geneva, WHO G...",0
2261,716,Zimbabwe,2019,Domestic general government health expenditure...,3.9,Data based on calendar year (January 1 to Dece...,"World Health Organization (WHO), Geneva, WHO G...",0


In [87]:

#con regex extrae de forma unique los valores que estan entre parebtesis en series 
Series = expenditure["Series"].str.extract(r"\((.*?)\)", expand=False)


for serie in Series.unique():
    expenditure[serie] = 0
    expenditure.loc[expenditure["Series"].str.contains(serie), serie] = 1

#eliminar columna series 
expenditure = expenditure.drop(columns=["Series"])

expenditure


,Region/Country/Area,Country Name,Year,Value,Footnotes,Source,% of GDP,% of total government expenditure
1,4,Afghanistan,2005,9.9,Data based on calendar year (January 1 to Dece...,"World Health Organization (WHO), Geneva, WHO G...",1,0
2,4,Afghanistan,2010,8.6,Data based on calendar year (January 1 to Dece...,"World Health Organization (WHO), Geneva, WHO G...",1,0
3,4,Afghanistan,2015,10.1,Data based on calendar year (January 1 to Dece...,"World Health Organization (WHO), Geneva, WHO G...",1,0
4,4,Afghanistan,2018,14.2,Data based on calendar year (January 1 to Dece...,"World Health Organization (WHO), Geneva, WHO G...",1,0
5,4,Afghanistan,2019,14.8,Data based on calendar year (January 1 to Dece...,"World Health Organization (WHO), Geneva, WHO G...",1,0
...,...,...,...,...,...,...,...,...
2258,716,Zimbabwe,2010,15.2,Data based on calendar year (January 1 to Dece...,"World Health Organization (WHO), Geneva, WHO G...",0,1
2259,716,Zimbabwe,2015,7.6,Data based on calendar year (January 1 to Dece...,"World Health Organization (WHO), Geneva, WHO G...",0,1
2260,716,Zimbabwe,2018,7.9,Data based on calendar year (January 1 to Dece...,"World Health Organization (WHO), Geneva, WHO G...",0,1
2261,716,Zimbabwe,2019,3.9,Data based on calendar year (January 1 to Dece...,"World Health Organization (WHO), Geneva, WHO G...",0,1


In [88]:
#eliminar footnotes y source 
expenditure = expenditure.drop(columns=["Footnotes"])
expenditure = expenditure.drop(columns=["Source"])



In [ ]:
#Unificar tablas conforme region/country/area y year

# Unir los dos DataFrames
merged = pd.merge(expenditure, Personnel, on=['Country Name', 'Year'], how='inner')

# Ver el resultado
merged

In [ ]:
for serie in series:
    # crear una columna para cada serie
    Personnel[serie] = Personnel["Series"].str.contains(serie).astype(int)

In [214]:
Personnel


,Region/Country/Area,Country Name,Year,Value,Source,number,per 1000 population,Physicians,Pharmacists,Nurses,Dentists
1,4,Afghanistan,2001,"4,104","World Health Organisation (WHO), Geneva, WHO G...",1,0,1,0,0,0
2,4,Afghanistan,2001,0.2,"World Health Organisation (WHO), Geneva, WHO G...",0,1,1,0,0,0
3,4,Afghanistan,2001,525,"World Health Organisation (WHO), Geneva, WHO G...",1,0,0,1,0,0
4,4,Afghanistan,2001,0.0,"World Health Organisation (WHO), Geneva, WHO G...",0,1,0,1,0,0
5,4,Afghanistan,2005,900,"World Health Organisation (WHO), Geneva, WHO G...",1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...
5880,716,Zimbabwe,2020,0.0,"World Health Organisation (WHO), Geneva, WHO G...",0,1,0,0,0,1
5881,716,Zimbabwe,2020,"1,600","World Health Organisation (WHO), Geneva, WHO G...",1,0,0,1,0,0
5882,716,Zimbabwe,2020,0.1,"World Health Organisation (WHO), Geneva, WHO G...",0,1,0,1,0,0
5883,716,Zimbabwe,2020,"31,792","World Health Organisation (WHO), Geneva, WHO G...",1,0,0,0,1,0


In [62]:
# dtypes de los dataframes
print("-----------Expenditure-----------")
print(expenditure.dtypes)
print("-----------Personnel-----------")
print(Personnel.dtypes)

-----------Expenditure-----------
1
4                                                                                                             object
Country Name                                                                                                  object
2005                                                                                                          object
Current health expenditure (% of GDP)                                                                         object
9.9                                                                                                           object
Data based on calendar year (January 1 to December 31).                                                       object
World Health Organization (WHO), Geneva, WHO Global Health Expenditure database, last accessed April 2023.    object
dtype: object
-----------Personnel-----------
1
4                                                                                                

In [92]:
# Tercer paso de limpieza de datos
# Cast de columnas
#expenditure["Region/Country/Area"].fillna(0, inplace=True)
#expenditure["Region/Country/Area"] = expenditure["Region/Country/Area"].astype(int)
#expenditure["Year"].fillna(2005, inplace=True)
#expenditure["Year"] = expenditure["Year"].astype(int)
#expenditure["Value"].fillna(0, inplace=True)
#expenditure["Value"] = expenditure["Value"].astype(float)

#Personnel["Region/Country/Area"].fillna(0, inplace=True)
#Personnel["Region/Country/Area"] = Personnel["Region/Country/Area"].astype(int)
#Personnel["Year"].fillna(2005, inplace=True)
#Personnel["Year"] = Personnel["Year"].astype(int)
#Personnel["Value"] = Personnel["Value"].str.replace(',', '.')
#Personnel["Value"].fillna(0.0, inplace=True)
#Personnel["Value"] = Personnel["Value"].astype(float)

,Region/Country/Area,Country Name,Year,Series,Value,Footnotes,Source
1,4,Afghanistan,2001,Health personnel: Physicians (number),"4,104",NaN,"World Health Organisation (WHO), Geneva, WHO G..."
2,4,Afghanistan,2001,Health personnel: Physicians (per 1000 populat...,0.2,NaN,"World Health Organisation (WHO), Geneva, WHO G..."
3,4,Afghanistan,2001,Health personnel: Pharmacists (number),525,NaN,"World Health Organisation (WHO), Geneva, WHO G..."
4,4,Afghanistan,2001,Health personnel: Pharmacists (per 1000 popula...,0.0,NaN,"World Health Organisation (WHO), Geneva, WHO G..."
5,4,Afghanistan,2005,Health personnel: Pharmacists (number),900,NaN,"World Health Organisation (WHO), Geneva, WHO G..."
...,...,...,...,...,...,...,...
5880,716,Zimbabwe,2020,Health personnel: Dentists (per 1000 population),0.0,NaN,"World Health Organisation (WHO), Geneva, WHO G..."
5881,716,Zimbabwe,2020,Health personnel: Pharmacists (number),"1,600",NaN,"World Health Organisation (WHO), Geneva, WHO G..."
5882,716,Zimbabwe,2020,Health personnel: Pharmacists (per 1000 popula...,0.1,NaN,"World Health Organisation (WHO), Geneva, WHO G..."
5883,716,Zimbabwe,2020,Health personnel: Nurses and midwives (number),"31,792",NaN,"World Health Organisation (WHO), Geneva, WHO G..."
